# star_plate.csv の生成

ここでは star_plate.csv の生成について考える。今回は実験的なもので、等間隔に星を散らばらせようと思う。これまでのデータを見ると、星は 25 pix くらい距離が離れていれば重ならなさそうにみえる。検出器は 1920 pix のフォーマットなので、 一辺アタリ 70 天体程度天体を置くことができる。しかし 70 x 70 も天体を置くと膨大な計算時間が必要となるため、せいぜい 20 x 20 程度にすることを考える。このくらいの個数で格子状に天体を配置することを考える。

star_plate.csv に必要となるのは plate index, star index, x pixel, y pixel, lambda, beta, Hwmag, nphoton である。が、現在は plate index = 0, star index 適当、x/y pixel グリッド状配置、Hwmag は後で適当に変えることにして 12.0 mag、などにして後は適当な値を入れておけばよい。適当な値はわかりやすく適当な値を入れよう。

ということで、一辺あたりの天体数を $n_{obj}$ として、適当な star_plate.csv を生成する。検出器フォーマットは $n_{pix}$ としよう。x/y pixel は完全なグリッドにすると PSF のピクセルへの載り方が一辺倒になるので、適当な乱数でばらしてみる。乱数のシードは適当に設定できるようにしよう。という感じで star_plate.csv を生成する関数を下記の通り定義。

In [1]:
import numpy as np

def mk_spcsv(nobj, npix, seedx, seedy, mag, filename):
    dpix = int(npix/nobj)
    py, px = (np.indices((nobj, nobj))+0.5)*dpix
    py = np.array(py, dtype='int').flatten()
    px = np.array(px, dtype='int').flatten()

    rng = np.random.default_rng(seedx)
    dpx = rng.normal(loc=0., scale=1.0, size=np.size(px))

    rng = np.random.default_rng(seedy)
    dpy = rng.normal(loc=0., scale=1.0, size=np.size(py))

    px = px + dpx
    py = py + dpy

    pl_idx = np.zeros_like(px)
    st_idx = np.arange(0,np.size(px))
    lmd = np.ones_like(px) * 999
    bet = np.ones_like(px) * 999
    hwm = np.ones_like(px) * mag
    nph = np.ones_like(px) * 999
    
    np.savetxt(filename, np.array([pl_idx, st_idx, px, py, lmd, bet, hwm, nph]).T, fmt="%d,%d,%e,%e,%.3f,%.3f,%.1f,%.1f",\
              header="plate index,star index,x pixel,y pixel,lambda,beta,Hwmag,nphoton", comments="")
    
    return

それでは生成してみる。

In [2]:
for nobj in [20]:
    for ds in range(0,10):
        seedx = 10+ds
        seedy = 20+ds
        for mag in [12.0]:
            mk_spcsv(nobj, 1920, seedx, seedy, mag, "spcsv_{:02d}_1920_{}_{}_{}.csv".format(nobj, seedx, seedy, mag))